# IA25_P01_G18 - Class Timetable

## Introdução
O objetivo deste projeto é desenvolver um agente inteligente para a geração de horários de aulas,formulado como um problema de satisfação de restrições (Constraint Satisfaction Problem - CSP).


### Equipa
- André Ferreira (a20764)

## Imports e configuração

In [29]:
from pathlib import Path
from itertools import islice
import time
from typing import Dict, List, Set

from constraint import Problem, MinConflictsSolver, AllDifferentConstraint

DATA_PATH = Path("data/ClassTT_01_tiny.txt")   
SLOTS = list(range(1, 21))                    
ROOMS_ALL = ["Lab01", "R101", "R102", "ONLINE"]
LESSON_INDEXES = [1, 2]                       


## Funções auxiliares

In [30]:
def day_of(slot: int) -> int:
    return (slot - 1) // 4

def block_in_day(slot: int) -> int:
    return (slot - 1) % 4

def parse_block(lines, header_prefix):
    try:
        start = next(i for i, l in enumerate(lines) if l.strip().startswith(header_prefix))
    except StopIteration:
        return []
    body = []
    for l in lines[start+1:]:
        if l.strip().startswith("#"):
            break
        if l.strip():
            body.append(l.strip())
    return body


## Carregar dataset e construir estruturas

In [31]:
raw = DATA_PATH.read_text(encoding="utf-8", errors="ignore").splitlines()

cc  = parse_block(raw, "#cc")  
dsd = parse_block(raw, "#dsd") 
tr  = parse_block(raw, "#tr")  
rr  = parse_block(raw, "#rr")  
oc  = parse_block(raw, "#oc")  

# classes -> cursos
COURSES_BY_CLASS: Dict[str, List[str]] = {}
for row in cc:
    parts = row.split()
    COURSES_BY_CLASS[parts[0]] = parts[1:]

CLASSES = sorted(COURSES_BY_CLASS.keys())
ALL_COURSES = sorted({c for lst in COURSES_BY_CLASS.values() for c in lst})

# docente -> cursos  e curso -> docente
TEACHER_COURSES: Dict[str, List[str]] = {}
TEACHER_BY_COURSE: Dict[str, str] = {}
for row in dsd:
    parts = row.split()
    t, cs = parts[0], parts[1:]
    TEACHER_COURSES[t] = cs
    for c in cs:
        TEACHER_BY_COURSE[c] = t

# indisponibilidades por docente
UNAVAILABLE: Dict[str, Set[int]] = {}
for row in tr:
    parts = row.split()
    t, slots = parts[0], list(map(int, parts[1:]))
    UNAVAILABLE[t] = set(slots)

# sala obrigatória por curso
ROOM_REQUIRED: Dict[str, str] = {}
for row in rr:
    c, r = row.split()
    ROOM_REQUIRED[c] = r

# aula online por índice
ONLINE_IDX: Dict[str, Set[int]] = {}
for row in oc:
    c, idx = row.split()
    ONLINE_IDX.setdefault(c, set()).add(int(idx))

print(f"Classes: {CLASSES}")
print(f"N.º total de cursos: {len(ALL_COURSES)}")
print(f"Docentes: {list(TEACHER_COURSES.keys())}")


Classes: ['t01', 't02', 't03']
N.º total de cursos: 15
Docentes: ['jo', 'mike', 'rob', 'sue']


## CSP: variáveis e domínios

In [32]:
problem = Problem(MinConflictsSolver())

for c in ALL_COURSES:
    teacher = TEACHER_BY_COURSE[c]
    blocked = UNAVAILABLE.get(teacher, set())
    allowed_slots = [s for s in SLOTS if s not in blocked]  

    for i in LESSON_INDEXES:
        problem.addVariable(f"{c}_{i}", allowed_slots)

        if c in ROOM_REQUIRED:
            problem.addVariable(f"{c}_{i}_room", [ROOM_REQUIRED[c]])
        elif c in ONLINE_IDX and i in ONLINE_IDX[c]:
            problem.addVariable(f"{c}_{i}_room", ["ONLINE"])
        else:
            problem.addVariable(f"{c}_{i}_room", ROOMS_ALL)


## Restrições

In [33]:
# 1) Aula online obrigatória quando marcado no dataset
for c, idxs in ONLINE_IDX.items():
    for i in idxs:
        problem.addConstraint(lambda r: r == "ONLINE", (f"{c}_{i}_room",))

# 2) Proibir 'ONLINE' quando não estiver marcado
for c in ALL_COURSES:
    for i in (1, 2):
        if not (c in ONLINE_IDX and i in ONLINE_IDX[c]):
            problem.addConstraint(lambda r: r != "ONLINE", (f"{c}_{i}_room",))

# 3) Sala obrigatória por curso 
for c, room in ROOM_REQUIRED.items():
    for i in LESSON_INDEXES:
        problem.addConstraint(lambda r, req=room: r == req, (f"{c}_{i}_room",))

# 4) Não sobrepor DOCENTE 
for t, courses in TEACHER_COURSES.items():
    slot_vars = [f"{c}_{i}" for c in courses for i in LESSON_INDEXES]
    if len(slot_vars) > 1:
        problem.addConstraint(AllDifferentConstraint(), slot_vars)

# 5) Não sobrepor TURMA 
for cls, courses in COURSES_BY_CLASS.items():
    slot_vars = [f"{c}_{i}" for c in courses for i in LESSON_INDEXES]
    if len(slot_vars) > 1:
        problem.addConstraint(AllDifferentConstraint(), slot_vars)

# 6) Máximo 3 aulas por dia por TURMA
def max3_lessons_per_day_for_class(problem, class_name, courses):
    slot_vars = [f"{c}_{i}" for c in courses for i in (1, 2)]
    def ok_max3_per_day(*slots):
        per_day = {d: 0 for d in range(5)}  
        for s in slots:
            per_day[day_of(s)] += 1
            if per_day[day_of(s)] > 3:
                return False
        return True
    if slot_vars:
        problem.addConstraint(ok_max3_per_day, tuple(slot_vars))

for cls, courses in COURSES_BY_CLASS.items():
    max3_lessons_per_day_for_class(problem, cls, courses)

# 7)  Se houver aulas ONLINE numa turma, têm de ser no mesmo dia e no MAX 3
def online_same_day_max3_for_class(problem, class_name, courses):
    online_slot_vars = []
    for c in courses:
        idxs = ONLINE_IDX.get(c, set())
        for i in idxs:
            online_slot_vars.append(f"{c}_{i}")  
    if len(online_slot_vars) <= 1:
        return  
    def same_day_max3(*slots):
        days = [day_of(s) for s in slots]
        return (len(set(days)) == 1) and (len(slots) <= 3)
    problem.addConstraint(same_day_max3, tuple(online_slot_vars))

for cls, courses in COURSES_BY_CLASS.items():
    online_same_day_max3_for_class(problem, cls, courses)


## Tempo a encontrar a primeira solução

In [34]:
start = time.perf_counter()
solution = problem.getSolution() 
elapsed = time.perf_counter() - start

if solution:
    print(f"Solução encontrada em {elapsed:.3f}s")
else:
    print("Nenhuma solução encontrada")


Solução encontrada em 0.004s


## Impressão da solução em tabela

In [35]:
def print_solution_table(sol, courses):
    print(f"{'Curso':<8} | {'Aula1(slot,sala)':<22} | {'Aula2(slot,sala)':<22}")
    print("-"*60)
    for c in courses:
        s1, r1 = sol[f"{c}_1"], sol[f"{c}_1_room"]
        s2, r2 = sol[f"{c}_2"], sol[f"{c}_2_room"]
        print(f"{c:<8} | ({s1:>2}, {r1:<12})     | ({s2:>2}, {r2:<12})")

if solution:
    print_solution_table(solution, ALL_COURSES)


Curso    | Aula1(slot,sala)       | Aula2(slot,sala)      
------------------------------------------------------------
UC11     | ( 5, Lab01       )     | (20, R101        )
UC12     | (12, R101        )     | (11, R101        )
UC13     | (13, Lab01       )     | ( 9, Lab01       )
UC14     | (16, Lab01       )     | (14, Lab01       )
UC15     | ( 6, R102        )     | ( 4, R102        )
UC21     | ( 8, R101        )     | (14, ONLINE      )
UC22     | ( 6, Lab01       )     | (18, Lab01       )
UC23     | ( 1, Lab01       )     | ( 9, R101        )
UC24     | (19, Lab01       )     | (12, R101        )
UC25     | (13, R101        )     | ( 2, R102        )
UC31     | ( 4, R101        )     | ( 7, ONLINE      )
UC32     | (10, Lab01       )     | ( 2, Lab01       )
UC33     | (15, R101        )     | ( 6, R102        )
UC34     | ( 5, R101        )     | (16, R102        )
UC35     | (14, R102        )     | ( 1, Lab01       )
